In [1]:

from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
import time
import codecs
import pandas as pd
import xml.etree.ElementTree as ET
import pymarc
from pymarc import Record,Field
import marcx
import io
from lxml import etree
import json
import glob
#from spst import SPST
import os
os.chdir('/Users/jammerman/Google Drive/git/hathitrust')
import string
#os.listdir('.')

In [2]:
def get_oclc(rec):
    _035s = rec.findall('*/[@tag="035"]')
    for _035 in _035s:

        try:
            _035a = _035.find('*/[@code="a"]').text
            if '(OCoLC)' in _035a:
                return(_035a)
            if _035a.isdigit():
                return(_035a)
        except Exception as e:
            pass    
    return(False)
    
def get_mtype(rec):
    leader = rec.find('./leader').text
    mtype = leader[6:8]
    return(mtype)

def get_mms_id(rec):
    mms_id = rec.find('*/[@tag="001"]').text
    return(mms_id)

def get_issn(rec):
    _022s = rec.findall('*/[@tag="022"]')
    for _022 in _022s:
        try:
            _022a = _022.find('*/[@code="a"]').text
            if '-' in _022a:
                return(_022a)
            if _022a.isdigit():
                return(_022a)
        except Exception as e:
            pass    
    return('')

def get_govdoc(rec):
    try:
        _008 = rec.find('*/[@tag="008"]').text
        if _008[28] == 'f':
            return('1')
    except Exception as e:
        pass
    return('0')

def get_holding_status(_949):
    try:
        _949g = _949.find('*/[@code="g"]').text
        #print(_949g)
        if 'BU' in _949g: ## in the BU Legacy Library
            #print('Withdrawn')
            return('WD')
    except Exception as e:
        pass   
    try:
        _949f = _949.find('*/[@code="f"]').text ## Missing or Lost status
        if 'MISSING' in _949f:
            #print('missing')
            return('LM')
        if 'LOST_LOAN' in _949f:
            #print('lost')
            return('LM')
    except Exception as e:
        #print(ET.dump(_949))
        #print()
        pass
    return''
def get_ser_fields(rec):
    ser_string = ''.join(get_oclc(rec)) + '\t'
    ser_string += get_mms_id(rec) + '\t'
    ser_string += get_issn(rec) + '\t'
    ser_string += get_govdoc((rec)) + '\n'
    return(ser_string)

def get_spm_fields(rec):
    _949s = rec.findall('*/[@tag="949"]')
    spm_string = ''
    for _949 in _949s:
        spm_string += get_oclc(rec) + '\t'
        spm_string += get_mms_id(rec) + '\t'
        spm_string += get_holding_status(_949) + '\t'
        spm_string += '\t' ## Condition not reportable
        spm_string += get_govdoc(rec) + '\n'
    return(spm_string)

def get_mpm_fields(rec):
    mpm_string = ''
    mpm_string += get_oclc(rec) + '\t'
    mpm_string += get_mms_id(rec) + '\t'
    mpm_string += get_holding_status(rec) + '\t'
    mpm_string += '\t' ## Condition not reportable
    _949s = rec.findall('*/[@tag="949"]')
    vols = []
    for _949 in _949s:
        try:
            _949c = _949.find('*/[@code="c"]').text
            vols.append(_949c)
        except Exception as e:
            pass  
    vols.sort()
    enum = ''.join(str(e) +'; ' for e in vols)
    mpm_string += enum +'\t' 
    mpm_string += get_govdoc(rec) + '\n'
    return(mpm_string)

def is_mpm(rec):
    _300s = rec.findall('*/[@tag="300"]')
    for _300 in _300s:
        try:
            _300a = _300.find('*/[@code="a"]').text
            if 'volumes' in _300a:
                return(True)
        except Exception as e:
            pass   
    
    _949s = rec.findall('*/[@tag="949"]')
    for _949 in _949s:
        try:
            _949a = _949.find('*/[@code="a"]').text
            if 'v.' in _949a:
                return(True)
        except Exception as e:
            pass   
    return(False)

In [3]:
file_list = glob.glob('2016/ht_*.xml')
ser_out = io.open('bu_serials_20160629.tsv','w')
spm_out = io.open('bu_single-part_20160629.tsv','w')
mpm_out = io.open('bu_multi-part_20160629.tsv','w')
err_out = io.open('bu_err_20160629.tsv','w')
mtypes = {}
fcount = 0
ser = 0
spm = 0
mpm = 0
acceptable = {'aa','ab','ac','am','ai','as'}
for f in file_list[0:]:
    print(str(fcount),f)
    fcount += 1
    tree = ET.parse(f)
    collection = tree.getroot()
    records = collection.findall('./record')
    for record in records[0:]:
        #print(ET.tostring(record))
        oclc_num = get_oclc(record)
        if oclc_num != False:
            mtype = get_mtype(record)
            if mtype in acceptable:
                if mtype in mtypes.keys():
                    mtypes[mtype] += 1
                else:
                    mtypes[mtype] = 1
                if mtype == 'as': ## we have a serial
                    #print(get_mms_id(record))
                    fields = get_ser_fields(record)
                    ser_out.write(fields)
                    ser += 1
                    #print(fields)
                if mtype == 'am': ## we have a monograph
                    if is_mpm(record) == True:
                        fields = get_mpm_fields(record)
                        mpm_out.write(fields)
                        mpm += 1
                    else:
                        fields = get_spm_fields(record)
                        spm_out.write(fields)
                        spm += 1
            else:
                mms_id = get_mms_id(record)
                err_out.write(mms_id + '\t' + oclc_num + '\n')
ser_out.close()
spm_out.close()
mpm_out.close()
err_out.close()

0 2016/ht_mono_2016062918_20538543220001161_new_1.xml
1 2016/ht_mono_2016062918_20538543220001161_new_10.xml
2 2016/ht_mono_2016062918_20538543220001161_new_100.xml
3 2016/ht_mono_2016062918_20538543220001161_new_101.xml
4 2016/ht_mono_2016062918_20538543220001161_new_102.xml
5 2016/ht_mono_2016062918_20538543220001161_new_103.xml
6 2016/ht_mono_2016062918_20538543220001161_new_104.xml
7 2016/ht_mono_2016062918_20538543220001161_new_105.xml
8 2016/ht_mono_2016062918_20538543220001161_new_106.xml
9 2016/ht_mono_2016062918_20538543220001161_new_107.xml
10 2016/ht_mono_2016062918_20538543220001161_new_108.xml
11 2016/ht_mono_2016062918_20538543220001161_new_109.xml
12 2016/ht_mono_2016062918_20538543220001161_new_11.xml
13 2016/ht_mono_2016062918_20538543220001161_new_110.xml
14 2016/ht_mono_2016062918_20538543220001161_new_111.xml
15 2016/ht_mono_2016062918_20538543220001161_new_112.xml
16 2016/ht_mono_2016062918_20538543220001161_new_113.xml
17 2016/ht_mono_2016062918_20538543220001161_

In [4]:
print(ser)
print(mpm)
print(spm)

33004
31600
1156147


In [5]:
print(mtypes)

{'ai': 976, 'aa': 364, 'am': 1187747, 'as': 33004, 'ac': 188}
